In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
import numpy as np
import time

In [2]:
from utils import check_if_titles_match, lower_and_remove_diacritics

Already scraped
artist_name = "Severina"
artist_name = "Mia Dimšić"
artist_name = "Jelena Rozga"

In [3]:
artist_name = "Petar Grašo"

In [4]:
df = pd.read_csv(f"data/{artist_name}_tekstovinet_lyricstranslate.csv")


In [5]:
df.head()

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,Album_lyricstranslate,Lyrics_lyricstranslate,EngLyrics_lyricstranslate,Url_lyricstranslate
0,nkaHNg1X_1_0,Petar Grašo,Petar Grašo,92,56927,Otkad nema te tu \r\nsve duzi je dan\r\nsve te...,"https://tekstovi.net/2,221,5620.html",NaN,'92,'92,Utorak,Otkad nema te tu\n1\nsve duži je dan\nsve teže...,Ever since you're not there\nThe day keeps get...,https://lyricstranslate.com/en/92-92.html-0
1,nkaHNg1X_1_1,Petar Grašo,Petar Grašo,Ako se rastanemo,26072,Ne govorim s tobom\r\ni nista nije kao prije\r...,"https://tekstovi.net/2,221,5621.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,nkaHNg1X_1_2,Petar Grašo,Petar Grašo,Ako te pitaju,185913,"Jos su do lani\r\nmoje noci, moji dani \r\nbil...","https://tekstovi.net/2,221,55130.html","Tonci Huljic, Vjekoslava Huljic, Petar Graso, ...",Ako te pitaju,If They Ask You,NaN,"Još su do lani moje noći, moji dani\n1\nBili k...","Until last year still my nights, my days\nWere...",https://lyricstranslate.com/en/ako-te-pitaju-i...
3,nkaHNg1X_1_3,Petar Grašo,Petar Grašo,Baraba,14089,Kazi mi s kim si \r\nnocas cekala zoru\r\nkome...,"https://tekstovi.net/2,221,5622.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nkaHNg1X_1_4,Petar Grašo,Petar Grašo,Brod ljubavi,27037,"Iznad tihog oceana\r\nbura, nevera, orkana\r\n...","https://tekstovi.net/2,221,5623.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df["Lyrics_lyricstranslate"].isnull().sum()

22

In [7]:
df[df["Lyrics_lyricstranslate"].isnull()]

,Song_ID,Artist,Artist_diskografija,Song_tekstovinet,Views_tekstovinet,Lyrics_tekstovinet,Url_tekstovinet,Writer_lyricstranslate,Song_lyricstranslate,EngTitle_lyricstranslate,Album_lyricstranslate,Lyrics_lyricstranslate,EngLyrics_lyricstranslate,Url_lyricstranslate
1,nkaHNg1X_1_1,Petar Grašo,Petar Grašo,Ako se rastanemo,26072,Ne govorim s tobom\r\ni nista nije kao prije\r...,"https://tekstovi.net/2,221,5621.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,nkaHNg1X_1_3,Petar Grašo,Petar Grašo,Baraba,14089,Kazi mi s kim si \r\nnocas cekala zoru\r\nkome...,"https://tekstovi.net/2,221,5622.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,nkaHNg1X_1_4,Petar Grašo,Petar Grašo,Brod ljubavi,27037,"Iznad tihog oceana\r\nbura, nevera, orkana\r\n...","https://tekstovi.net/2,221,5623.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,nkaHNg1X_1_11,Petar Grašo,Petar Grašo,Jedina,61982,"Jedina, jos te ima posvuda \r\nsvaki dan je os...","https://tekstovi.net/2,221,5674.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,nkaHNg1X_1_13,Petar Grašo,Petar Grašo,Kad ostarim,28864,Ne vjeruj \r\nKad kazu da sam lud \r\nDa me zn...,"https://tekstovi.net/2,221,2698.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,nkaHNg1X_1_14,Petar Grašo,Petar Grašo,Kada krenu vlakovi,19581,"Lako tebi, ti se uvijek\r\nNekom vratis\r\nI b...","https://tekstovi.net/2,221,2491.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,nkaHNg1X_1_16,Petar Grašo,Petar Grašo,Lakse mi je s' njom,32164,Lakse mi je s njom\r\nnego sa tobom \r\ndala m...,"https://tekstovi.net/2,221,5679.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,nkaHNg1X_1_17,Petar Grašo,Petar Grašo,Ljubav jedne zene,55329,Nebo moje ljubavi\r\nprekrila je siva boja \r\...,"https://tekstovi.net/2,221,5680.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,nkaHNg1X_1_19,Petar Grašo,Petar Grašo,Lomi me,33110,Godine su pomele \r\nvragove i andjele\r\nlipo...,"https://tekstovi.net/2,221,5675.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,nkaHNg1X_1_22,Petar Grašo,Petar Grašo,"More i ja, more i ti",16151,Pobit' cu nocas u note i stih \r\njer jos te i...,"https://tekstovi.net/2,221,2699.html",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
def scrape_for_authors_and_albums(song_name, artist_name):
    #search
    driver.find_element(By.CLASS_NAME, 'search-page').clear()
    search_box = driver.find_element(By.CLASS_NAME, 'search-page')
    song_name = lower_and_remove_diacritics(song_name)
    search_box.send_keys(song_name)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)
    
    
    #get results of search
    #search_result = driver.find_element(By.CLASS_NAME, 'search-result')
    try:
        song_results = driver.find_element(By.XPATH, f"//*[contains(text(), 'Pjesme')]/following-sibling::ul")
        song_results = song_results.find_elements(By.TAG_NAME, 'a')

        #go through search results
        for song in song_results:

            song.send_keys(Keys.CONTROL + Keys.RETURN)
            window_handles = driver.window_handles

            driver.switch_to.window(driver.window_handles[1])
            time.sleep(1)            

            if  check_performer(artist_name):
                subfields, authors = get_authors()
                albums = get_albums()
                song_url = driver.current_url
                song_name_website = driver.find_element(By.XPATH, f"//*[contains(text(), 'Pjesma')]/following-sibling::h1").text
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
                return subfields, authors, albums, song_url, song_name_website
                
            else:
                driver.close()
                driver.switch_to.window(window_name=window_handles[0])
    
    except Exception as e: print(f"Problem with scraping authors and albums for song {song_name}")

    return False    


In [9]:
def check_performer(artist_name):
    authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    artist_name = lower_and_remove_diacritics(artist_name)
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        if subfield == "Izvedba":
            siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current] 
            
    return True if len([r for r in result if artist_name in lower_and_remove_diacritics(r)])>0 else False


In [10]:
def get_authors():
    all_fields = []
    all_authors = []
    #authors = driver.find_elements(By.TAG_NAME, "h3")[1:]
    authors = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Autori')]/following-sibling::h3")
    for index, author in enumerate(authors):
        subfield = author.text.capitalize()
        siblings_of_current = driver.find_elements(By.XPATH, f"//*[contains(text(), '{subfield}')]/following-sibling::a")
        if len(siblings_of_current)>0:
            if index<len(authors)-1: 
                siblings_of_next = driver.find_elements(By.XPATH, f"//*[contains(text(), '{authors[index+1].text.capitalize()}')]/following-sibling::a")
                result = [r.text for r in siblings_of_current[0: len(siblings_of_current)-len(siblings_of_next)]]
            else: result = [r.text for r in siblings_of_current]
            all_fields.append(subfield)
            all_authors.append(str(result))
            
    return all_fields, all_authors


In [11]:
def get_albums():
    albums = driver.find_elements(By.XPATH, f"//*[contains(text(), 'Albumi')]/following-sibling::div")
    #return [f"{album.text}-{album.find_element(By.TAG_NAME, 'a').text}" for album in albums]
    return [{album.text} for album in albums]      
    

In [12]:
ser = Service("chromedriver.exe")
op = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=ser, options=op)
driver.get("https://diskografija.com/")


df_new = df.copy()#[0:5]
for row in df_new.iterrows():
    artist_name = row[1].Artist_diskografija
    song_name = row[1].Song_lyricstranslate
    if type(song_name)==float: song_name = row[1].Song_tekstovinet
    try: song_name = song_name[0:song_name.index("(")]
    except:pass
    
    try: 
        subfields, authors, albums, song_url, song_title_website = scrape_for_authors_and_albums(song_name, artist_name)
    
        for subfield, author in zip(subfields, authors):
            df.loc[row[0], f"{subfield}_diskografija"] = author
            
        for i, album in enumerate(albums):
            df.loc[row[0], f"Album_{i+1}_diskografija"] = album
        df.loc[row[0], "Url_diskografija"] = song_url
        df.loc[row[0], "Song_diskografija"] = song_title_website

        print(f"Succesfully found information for {song_name} from {artist_name}")
    
    except Exception as e:
        print(f"Problem with finding info for {song_name} from {artist_name}")
        print(e)
    print("#######")
    time.sleep(3)
    


Succesfully found information for '92 from Petar Grašo
#######
Succesfully found information for Ako se rastanemo from Petar Grašo
#######
Problem with finding info for Ako te pitaju from Petar Grašo
cannot unpack non-iterable bool object
#######
Succesfully found information for Baraba from Petar Grašo
#######
Succesfully found information for Brod ljubavi from Petar Grašo
#######
Problem with scraping authors and albums for song da li i'ko navija za nas
Problem with finding info for Da li i'ko navija za nas from Petar Grašo
cannot unpack non-iterable bool object
#######
Problem with finding info for Fritula from Petar Grašo
cannot unpack non-iterable bool object
#######
Succesfully found information for Godine ljubavi from Petar Grašo
#######
Succesfully found information for Gol, bos, sam from Petar Grašo
#######
Succesfully found information for Idi from Petar Grašo
#######
Problem with scraping authors and albums for song jel' ti reka 'ko
Problem with finding info for Jel' ti reka

In [13]:
check_if_titles_match(df, "Song_tekstovinet", "Song_diskografija")

,Song_tekstovinet,Song_diskografija
0,92,'92
16,lakse mi je s' njom,lakse mi je s njom
18,ljubav pozlati sve,ljubav sve pozlati
27,nek' pisma ostane,nek pisma ostane


In [14]:
df["Lyrics_lyricstranslate"].isnull().sum()

22

In [15]:
df.to_csv(f"data/{artist_name}_tekstovinet_lyricstranslate_diskografija.csv", index=False)
